In [ ]:
import numpy as np
from pyFTS import *
from pyFTS.common import Membership


class FuzzySet(object):
    """
    Fuzzy Set
    """

    def __init__(self, name, mf, parameters, centroid, alpha=1.0, **kwargs):
        """
        Create a Fuzzy Set
        """
        self.name = name
        """The fuzzy set name"""
        self.mf = mf
        """The membership function"""
        self.parameters = parameters
        """The parameters of the membership function"""
        self.centroid = centroid
        """The fuzzy set center of mass (or midpoint)"""
        self.alpha = alpha
        """The alpha cut value"""
        self.type = kwargs.get('type', 'common')
        """The fuzzy set type (common, composite, nonstationary, etc)"""
        self.variable = kwargs.get('variable', None)
        """In multivariate time series, indicate for which variable this fuzzy set belogs"""
        self.Z = None
        """Partition function in respect to the membership function"""

        if parameters is not None:
            if self.mf == Membership.gaussmf:
                self.lower = parameters[0] - parameters[1] * 3
                self.upper = parameters[0] + parameters[1] * 3
            elif self.mf == Membership.sigmf:
                k = (parameters[1] / (2 * parameters[0]))
                self.lower = parameters[1] - k
                self.upper = parameters[1] + k
            else:
                self.lower = min(parameters)
                self.upper = max(parameters)

        self.metadata = {}


    def transform(self, x):
        """
        Preprocess the data point for non native types

        :param x:
        :return: return a native type value for the structured type
        """

        return x


    def membership(self, x):
        """
        Calculate the membership value of a given input

        :param x: input value 
        :return: membership value of x at this fuzzy set
        """
        return self.mf(self.transform(x), self.parameters) * self.alpha



    def partition_function(self, uod=None, nbins=100):
        """
        Calculate the partition function over the membership function.

        :param uod:
        :param nbins:
        :return:
        """
        if self.Z is None and uod is not None:
            self.Z = 0.0
            for k in np.linspace(uod[0], uod[1], nbins):
                self.Z += self.membership(k)

        return self.Z


    def __str__(self):
        return self.name + ": " + str(self.mf.__name__) + "(" + str(self.parameters) + ")"



def __binary_search(x, fuzzy_sets, ordered_sets):
    """
    Search for elegible fuzzy sets to fuzzyfy x

    :param x: input value to be fuzzyfied
    :param fuzzy_sets:  a dictionary where the key is the fuzzy set name and the value is the fuzzy set object.
    :param ordered_sets: a list with the fuzzy sets names ordered by their centroids.
    :return: A list with the best fuzzy sets that may contain x
    """
    max_len = len(fuzzy_sets) - 1
    first = 0
    last = max_len

    while first <= last:
        midpoint = (first + last) // 2

        fs = ordered_sets[midpoint]
        fs1 = ordered_sets[midpoint - 1] if midpoint > 0 else ordered_sets[0]
        fs2 = ordered_sets[midpoint + 1] if midpoint < max_len else ordered_sets[max_len]

        if fuzzy_sets[fs1].centroid <= fuzzy_sets[fs].transform(x) <= fuzzy_sets[fs2].centroid:
            return (midpoint - 1, midpoint, midpoint + 1)
        elif midpoint <= 1:
            return [0]
        elif midpoint >= max_len:
            return [max_len]
        else:
            if fuzzy_sets[fs].transform(x) < fuzzy_sets[fs].centroid:
                last = midpoint - 1
            else:
                first = midpoint + 1



def fuzzyfy(data, partitioner, **kwargs):
    """
    A general method for fuzzyfication.

    :param data: input value to be fuzzyfied
    :param partitioner: a trained pyFTS.partitioners.Partitioner object
    :param kwargs: dict, optional arguments
    :keyword alpha_cut: the minimal membership value to be considered on fuzzyfication (only for mode='sets')
    :keyword method: the fuzzyfication method (fuzzy: all fuzzy memberships, maximum: only the maximum membership)
    :keyword mode: the fuzzyfication mode (sets: return the fuzzy sets names, vector: return a vector with the membership
    values for all fuzzy sets, both: return a list with tuples (fuzzy set, membership value) )
    :returns a list with the fuzzyfied values, depending on the mode

    """
    alpha_cut = kwargs.get('alpha_cut', 0.)
    mode = kwargs.get('mode', 'sets')
    method = kwargs.get('method', 'fuzzy')
    if isinstance(data, (list, np.ndarray)):
        if mode == 'vector':
            return fuzzyfy_instances(data, partitioner.sets, partitioner.ordered_sets)
        elif mode == 'both':
            mvs = fuzzyfy_instances(data, partitioner.sets, partitioner.ordered_sets)
            fs = []
            for mv in mvs:
                fsets = [(partitioner.ordered_sets[ix], mv[ix])
                         for ix in np.arange(len(mv))
                         if mv[ix] >= alpha_cut]
                fs.append(fsets)
            return fs
        else:
            return fuzzyfy_series(data, partitioner.sets, method, alpha_cut, partitioner.ordered_sets)
    else:
        if mode == 'vector':
            return fuzzyfy_instance(data, partitioner.sets, partitioner.ordered_sets)
        elif mode == 'both':
            mv = fuzzyfy_instance(data, partitioner.sets, partitioner.ordered_sets)
            fsets = [(partitioner.ordered_sets[ix], mv[ix])
                     for ix in np.arange(len(mv))
                     if mv[ix] >= alpha_cut]
            return fsets
        else:
            return get_fuzzysets(data, partitioner.sets, partitioner.ordered_sets, alpha_cut)




def set_ordered(fuzzy_sets):
    """
    Order a fuzzy set list by their centroids

    :param fuzzy_sets: a dictionary where the key is the fuzzy set name and the value is the fuzzy set object.
    :return: a list with the fuzzy sets names ordered by their centroids.
    """
    if len(fuzzy_sets) > 0:
        tmp1 = [fuzzy_sets[k] for k in fuzzy_sets.keys()]
        return [k.name for k in sorted(tmp1, key=lambda x: x.centroid)]




def fuzzyfy_instance(inst, fuzzy_sets, ordered_sets=None):
    """
    Calculate the membership values for a data point given fuzzy sets

    :param inst: data point
    :param fuzzy_sets: a dictionary where the key is the fuzzy set name and the value is the fuzzy set object.
    :param ordered_sets: a list with the fuzzy sets names ordered by their centroids.
    :return: array of membership values
    """

    if ordered_sets is None:
        ordered_sets = set_ordered(fuzzy_sets)

    mv = np.zeros(len(fuzzy_sets))

    for ix in __binary_search(inst, fuzzy_sets, ordered_sets):
        mv[ix] = fuzzy_sets[ordered_sets[ix]].membership(inst)

    return mv



def fuzzyfy_instances(data, fuzzy_sets, ordered_sets=None):
    """
    Calculate the membership values for a data point given fuzzy sets

    :param inst: data point
    :param fuzzy_sets: a dictionary where the key is the fuzzy set name and the value is the fuzzy set object.
    :param ordered_sets: a list with the fuzzy sets names ordered by their centroids.
    :return: array of membership values
    """
    ret = []
    if ordered_sets is None:
        ordered_sets = set_ordered(fuzzy_sets)
    for inst in data:
        mv = fuzzyfy_instance(inst, fuzzy_sets, ordered_sets)
        ret.append(mv)
    return ret



def get_fuzzysets(inst, fuzzy_sets, ordered_sets=None, alpha_cut=0.0):
    """
    Return the fuzzy sets which membership value for a inst is greater than the alpha_cut

    :param inst: data point
    :param fuzzy_sets:  a dictionary where the key is the fuzzy set name and the value is the fuzzy set object.
    :param ordered_sets: a list with the fuzzy sets names ordered by their centroids.
    :param alpha_cut: Minimal membership to be considered on fuzzyfication process
    :return: array of membership values
    """

    if ordered_sets is None:
        ordered_sets = set_ordered(fuzzy_sets)

    try:
        fs = [ordered_sets[ix]
              for ix in __binary_search(inst, fuzzy_sets, ordered_sets)
              if fuzzy_sets[ordered_sets[ix]].membership(inst) > alpha_cut]
        return fs
    except Exception as ex:
        raise ex



def get_maximum_membership_fuzzyset(inst, fuzzy_sets, ordered_sets=None):
    """
    Fuzzify a data point, returning the fuzzy set with maximum membership value

    :param inst: data point
    :param fuzzy_sets:  a dictionary where the key is the fuzzy set name and the value is the fuzzy set object.
    :param ordered_sets: a list with the fuzzy sets names ordered by their centroids.
    :return: fuzzy set with maximum membership
    """
    if ordered_sets is None:
        ordered_sets = set_ordered(fuzzy_sets)
    mv = np.array([fuzzy_sets[key].membership(inst) for key in ordered_sets])
    key = ordered_sets[np.argwhere(mv == max(mv))[0, 0]]
    return fuzzy_sets[key]



def get_maximum_membership_fuzzyset_index(inst, fuzzy_sets):
    """
    Fuzzify a data point, returning the fuzzy set with maximum membership value

    :param inst: data point
    :param fuzzy_sets: dict of fuzzy sets
    :return: fuzzy set with maximum membership
    """
    mv = fuzzyfy_instance(inst, fuzzy_sets)
    return np.argwhere(mv == max(mv))[0, 0]



def fuzzyfy_series_old(data, fuzzy_sets, method='maximum'):
    fts = []
    for item in data:
        fts.append(get_maximum_membership_fuzzyset(item, fuzzy_sets).name)
    return fts



def fuzzyfy_series(data, fuzzy_sets, method='maximum', alpha_cut=0.0, ordered_sets=None):
    fts = []
    if ordered_sets is None:
        ordered_sets = set_ordered(fuzzy_sets)
    for t, i in enumerate(data):
        mv = fuzzyfy_instance(i, fuzzy_sets, ordered_sets)
        if len(mv) == 0:
            sets = check_bounds(i, fuzzy_sets.items(), ordered_sets)
        else:
            if method == 'fuzzy':
                ix = np.ravel(np.argwhere(mv > alpha_cut))
                sets = [fuzzy_sets[ordered_sets[i]].name for i in ix]
            elif method == 'maximum':
                mx = max(mv)
                ix = np.ravel(np.argwhere(mv == mx))
                sets = fuzzy_sets[ordered_sets[ix[0]]].name
        fts.append(sets)
    return fts




def grant_bounds(data, fuzzy_sets, ordered_sets):
    if data < fuzzy_sets[ordered_sets[0]].lower:
        return fuzzy_sets[ordered_sets[0]].lower
    elif data > fuzzy_sets[ordered_sets[-1]].upper:
        return fuzzy_sets[ordered_sets[-1]].upper
    else:
        return data



def check_bounds(data, fuzzy_sets, ordered_sets):
    if data < fuzzy_sets[ordered_sets[0]].lower:
        return fuzzy_sets[ordered_sets[0]]
    elif data > fuzzy_sets[ordered_sets[-1]].upper:
        return fuzzy_sets[ordered_sets[-1]]



def check_bounds_index(data, fuzzy_sets, ordered_sets):
    if data < fuzzy_sets[ordered_sets[0]].get_lower():
        return 0
    elif data > fuzzy_sets[ordered_sets[-1]].get_upper():
        return len(fuzzy_sets) - 1

In [ ]:
"""
Membership functions for Fuzzy Sets
"""

import numpy as np
import math
from pyFTS import *


def trimf(x, parameters):
    """
    Triangular fuzzy membership function

    :param x: data point
    :param parameters: a list with 3 real values
    :return: the membership value of x given the parameters
    """

    xx = np.round(x, 3)
    if xx < parameters[0]:
        return 0
    elif parameters[0] <= xx < parameters[1]:
        return (x - parameters[0]) / (parameters[1] - parameters[0])
    elif parameters[1] <= xx <= parameters[2]:
        return (parameters[2] - xx) / (parameters[2] - parameters[1])
    else:
        return 0



def trapmf(x, parameters):
    """
    Trapezoidal fuzzy membership function

    :param x: data point
    :param parameters: a list with 4 real values
    :return: the membership value of x given the parameters
    """

    if x < parameters[0]:
        return 0
    elif parameters[0] <= x < parameters[1]:
        return (x - parameters[0]) / (parameters[1] - parameters[0])
    elif parameters[1] <= x <= parameters[2]:
        return 1
    elif parameters[2] <= x <= parameters[3]:
        return (parameters[3] - x) / (parameters[3] - parameters[2])
    else:
        return 0




def gaussmf(x, parameters):
    """
    Gaussian fuzzy membership function

    :param x: data point
    :param parameters: a list with 2 real values (mean and variance)
    :return: the membership value of x given the parameters
    """

    return math.exp((-(x - parameters[0])**2)/(2 * parameters[1]**2))



def bellmf(x, parameters):
    """
    Bell shaped membership function

    :param x:
    :param parameters:
    :return:
    """

    return 1 / (1 + abs((x - parameters[2]) / parameters[0]) ** (2 * parameters[1]))



def sigmf(x, parameters):
    """
    Sigmoid / Logistic membership function

    :param x:
    :param parameters: an list with 2 real values (smoothness and midpoint)
    :return
    """
    return 1 / (1 + math.exp(-parameters[0] * (x - parameters[1])))



def singleton(x, parameters):
    """
    Singleton membership function, a single value fuzzy function

    :param x:
    :param parameters: a list with one real value
    :returns
    """
    return 1 if x == parameters[0] else 0

In [ ]:
import numpy as np
import pandas as pd
from pyFTS.common import FuzzySet, SortedCollection, tree, Util


class FTS(object):
    """
    Fuzzy Time Series object model
    """
    def __init__(self, **kwargs):
        """
        Create a Fuzzy Time Series model
        """
        self.flrgs = {}
        """The list of Fuzzy Logical Relationship Groups - FLRG"""
        self.order = kwargs.get('order',1)
        """A integer with the model order (number of past lags are used on forecasting)"""
        self.shortname = kwargs.get('name',"")
        """A string with a short name or alias for the model"""
        self.name = kwargs.get('name',"")
        """A string with the model name"""
        self.detail = kwargs.get('name',"")
        """A string with the model detailed information"""
        self.is_wrapper = False
        """Indicates that this model is a wrapper for other(s) method(s)"""
        self.is_high_order = False
        """A boolean value indicating if the model support orders greater than 1, default: False"""
        self.min_order = 1
        """In high order models, this integer value indicates the minimal order supported for the model, default: 1"""
        self.has_seasonality = False
        """A boolean value indicating if the model supports seasonal indexers, default: False"""
        self.has_point_forecasting = True
        """A boolean value indicating if the model supports point forecasting, default: True"""
        self.has_interval_forecasting = False
        """A boolean value indicating if the model supports interval forecasting, default: False"""
        self.has_probability_forecasting = False
        """A boolean value indicating if the model support probabilistic forecasting, default: False"""
        self.is_multivariate = False
        """A boolean value indicating if the model support multivariate time series (Pandas DataFrame), default: False"""
        self.is_clustered = False
        """A boolean value indicating if the model support multivariate time series (Pandas DataFrame), but works like 
        a monovariate method, default: False"""
        self.dump = False
        self.transformations = []
        """A list with the data transformations (common.Transformations) applied on model pre and post processing, default: []"""
        self.transformations_param = []
        """A list with the specific parameters for each data transformation"""
        self.original_max = 0
        """A float with the upper limit of the Universe of Discourse, the maximal value found on training data"""
        self.original_min = 0
        """A float with the lower limit of the Universe of Discourse, the minimal value found on training data"""
        self.partitioner = kwargs.get("partitioner", None)
        """A pyFTS.partitioners.Partitioner object with the Universe of Discourse partitioner used on the model. This is a mandatory dependecy. """
        if self.partitioner != None:
            self.sets = self.partitioner.sets
        self.auto_update = False
        """A boolean value indicating that model is incremental"""
        self.benchmark_only = False
        """A boolean value indicating a façade for external (non-FTS) model used on benchmarks or ensembles."""
        self.indexer = kwargs.get("indexer", None)
        """An pyFTS.models.seasonal.Indexer object for indexing the time series data"""
        self.uod_clip = kwargs.get("uod_clip", True)
        """Flag indicating if the test data will be clipped inside the training Universe of Discourse"""
        self.alpha_cut = kwargs.get("alpha_cut", 0.0)
        """A float with the minimal membership to be considered on fuzzyfication process"""
        self.lags = kwargs.get("lags", None)
        """The list of lag indexes for high order models"""
        self.max_lag = self.order
        """A integer indicating the largest lag used by the model. This value also indicates the minimum number of past lags 
        needed to forecast a single step ahead"""
        self.log = pd.DataFrame([],columns=["Datetime","Operation","Value"])
        """"""
        self.is_time_variant = False
        """A boolean value indicating if this model is time variant"""
        self.standard_horizon = kwargs.get("standard_horizon", 1)
        """Standard forecasting horizon (Default: 1)"""
        

    def fuzzy(self, data):
        """
        Fuzzify a data point

        :param data: data point
        :return: maximum membership fuzzy set
        """
        best = {"fuzzyset": "", "membership": 0.0}

        for f in self.partitioner.sets:
            fset = self.partitioner.sets[f]
            if best["membership"] <= fset.membership(data):
                best["fuzzyset"] = fset.name
                best["membership"] = fset.membership(data)

        return best


    def clip_uod(self, ndata):
        if self.uod_clip and self.partitioner is not None:
            ndata = np.clip(ndata, self.partitioner.min, self.partitioner.max)
        elif self.uod_clip:
            ndata = np.clip(ndata, self.original_min, self.original_max)
        return ndata


    def predict(self, data, **kwargs):
        """
        Forecast using trained model

        :param data: time series with minimal length to the order of the model

        :keyword type: the forecasting type, one of these values: point(default), interval, distribution or multivariate.
        :keyword steps_ahead: The forecasting path H, i. e., tell the model to forecast from t+1 to t+H.
        :keyword step_to: The forecasting step H, i. e., tell the model to forecast to t+H for each input sample 
        :keyword start_at: in the multi step forecasting, the index of the data where to start forecasting (default value: 0)
        :keyword distributed: boolean, indicate if the forecasting procedure will be distributed in a dispy cluster (default value: False)
        :keyword nodes: a list with the dispy cluster nodes addresses
        :keyword explain: try to explain, step by step, the one-step-ahead point forecasting result given the input data. (default value: False)
        :keyword generators: for multivariate methods on multi step ahead forecasting, generators is a dict where the keys
                            are the dataframe columun names (except the target_variable) and the values are lambda functions that
                            accept one value (the actual value of the variable) and return the next value or trained FTS
                            models that accept the actual values and forecast new ones.

        :return: a numpy array with the forecasted data
        """
        import copy

        kw = copy.deepcopy(kwargs)

        if self.is_multivariate:
            ndata = data
        else:
            ndata = self.apply_transformations(data)

        ndata = self.clip_uod(ndata)

        if 'distributed' in kw:
            distributed = kw.pop('distributed')
        else:
            distributed = False

        if 'type' in kw:
            type = kw.pop("type")
        else:
            type = 'point'

        if distributed is None or distributed == False:

            steps_ahead = kw.get("steps_ahead", None)

            step_to = kw.get("step_to", None)

            if (steps_ahead == None and step_to == None) or (steps_ahead == 1 or step_to ==1):
                if type == 'point':
                    ret = self.forecast(ndata, **kw)
                elif type == 'interval':
                    ret = self.forecast_interval(ndata, **kw)
                elif type == 'distribution':
                    ret = self.forecast_distribution(ndata, **kw)
                elif type == 'multivariate':
                    ret = self.forecast_multivariate(ndata, **kw)
            elif step_to == None and steps_ahead > 1:
                if type == 'point':
                    ret = self.forecast_ahead(ndata, steps_ahead, **kw)
                elif type == 'interval':
                    ret = self.forecast_ahead_interval(ndata, steps_ahead, **kw)
                elif type == 'distribution':
                    ret = self.forecast_ahead_distribution(ndata, steps_ahead, **kw)
                elif type == 'multivariate':
                    ret = self.forecast_ahead_multivariate(ndata, steps_ahead, **kw)
            elif step_to > 1:
                if type == 'point':
                    ret = self.forecast_step(ndata, step_to, **kw)
                else:
                    raise NotImplementedError('This model only perform point step ahead forecasts!')

            if not ['point', 'interval', 'distribution', 'multivariate'].__contains__(type):
                raise ValueError('The argument \'type\' has an unknown value.')

        else:

            if distributed == 'dispy':
                from pyFTS.distributed import dispy

                nodes = kw.pop("nodes", ['127.0.0.1'])
                num_batches = kw.pop('num_batches', 10)

                ret = dispy.distributed_predict(self, kw, nodes, ndata, num_batches, **kw)

            elif distributed == 'spark':
                from pyFTS.distributed import spark

                ret = spark.distributed_predict(data=ndata, model=self, **kw)

        if not self.is_multivariate:
            kw['type'] = type
            ret = self.apply_inverse_transformations(ret, params=[data[self.max_lag - 1:]], **kw)

        if 'statistics' in kw:
            kwargs['statistics'] = kw['statistics']

        return ret


    def forecast(self, data, **kwargs):
        """
        Point forecast one step ahead

        :param data: time series data with the minimal length equal to the max_lag of the model
        :param kwargs: model specific parameters
        :return: a list with the forecasted values
        """
        raise NotImplementedError('This model do not perform one step ahead point forecasts!')


    def forecast_interval(self, data, **kwargs):
        """
        Interval forecast one step ahead

        :param data: time series data with the minimal length equal to the max_lag of the model
        :param kwargs: model specific parameters
        :return: a list with the prediction intervals
        """
        raise NotImplementedError('This model do not perform one step ahead interval forecasts!')


    def forecast_distribution(self, data, **kwargs):
        """
        Probabilistic forecast one step ahead

        :param data: time series data with the minimal length equal to the max_lag of the model
        :param kwargs: model specific parameters
        :return: a list with probabilistic.ProbabilityDistribution objects representing the forecasted Probability Distributions
        """
        raise NotImplementedError('This model do not perform one step ahead distribution forecasts!')


    def forecast_multivariate(self, data, **kwargs):
        """
        Multivariate forecast one step ahead

        :param data: Pandas dataframe with one column for each variable and with the minimal length equal to the max_lag of the model
        :param kwargs: model specific parameters
        :return: a Pandas Dataframe object representing the forecasted values for each variable
        """
        raise NotImplementedError('This model do not perform one step ahead multivariate forecasts!')



    def forecast_ahead(self, data, steps, **kwargs):
        """
        Point forecast from 1 to H steps ahead, where H is given by the steps parameter

        :param data: time series data with the minimal length equal to the max_lag of the model
        :param steps: the number of steps ahead to forecast (default: 1)
        :keyword start_at: in the multi step forecasting, the index of the data where to start forecasting (default: 0)
        :return: a list with the forecasted values
        """

        if len(data) < self.max_lag:
            return data

        if isinstance(data, np.ndarray):
            data = data.tolist()

        start = kwargs.get('start_at',0)

        ret = data[:start+self.max_lag]
        for k in np.arange(start+self.max_lag, steps+start+self.max_lag):
            tmp = self.forecast(ret[k-self.max_lag:k], **kwargs)

            if isinstance(tmp,(list, np.ndarray)):
                tmp = tmp[-1]

            ret.append(tmp)
            data.append(tmp)

        return ret[-steps:]


    def forecast_ahead_interval(self, data, steps, **kwargs):
        """
        Interval forecast from 1 to H steps ahead, where H is given by the steps parameter

        :param data: time series data with the minimal length equal to the max_lag of the model
        :param steps: the number of steps ahead to forecast
        :keyword start_at: in the multi step forecasting, the index of the data where to start forecasting (default: 0)
        :return: a list with the forecasted intervals
        """
        raise NotImplementedError('This model do not perform multi step ahead interval forecasts!')


    def forecast_ahead_distribution(self, data, steps, **kwargs):
        """
        Probabilistic forecast from 1 to H steps ahead, where H is given by the steps parameter

        :param data: time series data with the minimal length equal to the max_lag of the model
        :param steps: the number of steps ahead to forecast
        :keyword start_at: in the multi step forecasting, the index of the data where to start forecasting (default: 0)
        :return: a list with the forecasted Probability Distributions
        """
        raise NotImplementedError('This model do not perform multi step ahead distribution forecasts!')


    def forecast_ahead_multivariate(self, data, steps, **kwargs):
        """
        Multivariate forecast n step ahead

        :param data: Pandas dataframe with one column for each variable and with the minimal length equal to the max_lag of the model
        :param steps: the number of steps ahead to forecast
        :keyword start_at: in the multi step forecasting, the index of the data where to start forecasting (default: 0)
        :return: a Pandas Dataframe object representing the forecasted values for each variable
        """
        raise NotImplementedError('This model do not perform one step ahead multivariate forecasts!')


    def forecast_step(self, data, step, **kwargs):
        """
        Point forecast for H steps ahead, where H is given by the step parameter

        :param data: time series data with the minimal length equal to the max_lag of the model
        :param step: the forecasting horizon (default: 1)
        :keyword start_at: in the multi step forecasting, the index of the data where to start forecasting (default: 0)
        :return: a list with the forecasted values
        """

        l = len(data)

        ret = []

        if l < self.max_lag:
            return data

        if isinstance(data, np.ndarray):
            data = data.tolist()

        start = kwargs.get('start_at',0)

        for k in np.arange(start+self.max_lag, l):
            sample = data[k-self.max_lag:k]
            tmp = self.forecast_ahead(sample, step, **kwargs)

            if isinstance(tmp,(list, np.ndarray)):
                tmp = tmp[-1]

            ret.append(tmp)

        return ret


    def train(self, data, **kwargs):
        """
        Method specific parameter fitting

        :param data: training time series data
        :param kwargs: Method specific parameters

        """
        pass


    def fit(self, ndata, **kwargs):
        """
        Fit the model's parameters based on the training data.

        :param ndata: training time series data
        :param kwargs:

        :keyword num_batches: split the training data in num_batches to save memory during the training process
        :keyword save_model: save final model on disk
        :keyword batch_save: save the model between each batch
        :keyword file_path: path to save the model
        :keyword distributed: boolean, indicate if the training procedure will be distributed in a dispy cluster
        :keyword nodes: a list with the dispy cluster nodes addresses

        """

        import datetime, copy

        kw = copy.deepcopy(kwargs)

        if self.is_multivariate:
            data = ndata
        else:
            data = self.apply_transformations(ndata)

            self.original_min = np.nanmin(data)
            self.original_max = np.nanmax(data)

        if 'partitioner' in kw:
            self.partitioner = kw.pop('partitioner')

        if not self.is_multivariate and not self.is_wrapper and not self.benchmark_only:
            if self.partitioner is None:
                raise Exception("Fuzzy sets were not provided for the model. Use 'partitioner' parameter. ")

        if 'order' in kw:
            self.order = kw.pop('order')

        dump = kw.get('dump', None)

        num_batches = kw.pop('num_batches', None)

        save = kw.get('save_model', False)  # save model on disk

        batch_save = kw.get('batch_save', False) #save model between batches

        file_path = kw.get('file_path', None)

        distributed = kw.pop('distributed', False)

        if distributed is not None and distributed:
            if num_batches is None:
                num_batches = 10

            if distributed == 'dispy':
                from pyFTS.distributed import dispy
                nodes = kw.pop('nodes', False)
                train_method = kwargs.get('train_method', dispy.simple_model_train)
                dispy.distributed_train(self, train_method, nodes, type(self), data, num_batches, {},
                                       **kw)
            elif distributed == 'spark':
                from pyFTS.distributed import spark
                url = kwargs.get('url', 'spark://127.0.0.1:7077')
                app = kwargs.get('app', 'pyFTS')

                spark.distributed_train(self, data, url=url, app=app)
        else:

            if dump == 'time':
                print("[{0: %H:%M:%S}] Start training".format(datetime.datetime.now()))

            if num_batches is not None and not self.is_wrapper:
                n = len(data)
                batch_size = int(n / num_batches)
                bcount = 1

                rng = range(self.order, n, batch_size)

                if dump == 'tqdm':
                    from tqdm import tqdm

                    rng = tqdm(rng)

                for ct in rng:
                    if dump == 'time':
                        print("[{0: %H:%M:%S}] Starting batch ".format(datetime.datetime.now()) + str(bcount))
                    if self.is_multivariate:
                        mdata = data.iloc[ct - self.order:ct + batch_size]
                    else:
                        mdata = data[ct - self.order : ct + batch_size]

                    self.train(mdata, **kw)

                    if batch_save:
                        Util.persist_obj(self,file_path)

                    if dump == 'time':
                        print("[{0: %H:%M:%S}] Finish batch ".format(datetime.datetime.now()) + str(bcount))

                    bcount += 1

            else:
                self.train(data, **kw)

            if dump == 'time':
                print("[{0: %H:%M:%S}] Finish training".format(datetime.datetime.now()))

        if save:
            Util.persist_obj(self, file_path)



    def clone_parameters(self, model):
        """
        Import the parameters values from other model

        :param model: a model to clone the parameters
        """

        self.order = model.order
        self.partitioner = model.partitioner
        self.lags = model.lags
        self.shortname = model.shortname
        self.name = model.name
        self.detail = model.detail
        self.is_high_order = model.is_high_order
        self.min_order = model.min_order
        self.has_seasonality = model.has_seasonality
        self.has_point_forecasting = model.has_point_forecasting
        self.has_interval_forecasting = model.has_interval_forecasting
        self.has_probability_forecasting = model.has_probability_forecasting
        self.is_multivariate = model.is_multivariate
        self.dump = model.dump
        self.transformations = model.transformations
        self.transformations_param = model.transformations_param
        self.original_max = model.original_max
        self.original_min = model.original_min
        self.auto_update = model.auto_update
        self.benchmark_only = model.benchmark_only
        self.indexer = model.indexer


    def append_rule(self, flrg):
        """
        Append FLRG rule to the model

        :param flrg: rule
        :return:
        """

        if flrg.get_key() not in self.flrgs:
            self.flrgs[flrg.get_key()] = flrg
        else:
            if isinstance(flrg.RHS, (list, set)):
                for k in flrg.RHS:
                    self.flrgs[flrg.get_key()].append_rhs(k)
            elif isinstance(flrg.RHS, dict):
                for key, value in flrg.RHS.items():
                    self.flrgs[flrg.get_key()].append_rhs(key, count=value)
            else:
                self.flrgs[flrg.get_key()].append_rhs(flrg.RHS)


    def merge(self, model):
        """
        Merge the FLRG rules from other model

        :param model: source model
        :return:
        """

        for key, flrg in model.flrgs.items():
            self.append_rule(flrg)


    def append_transformation(self, transformation):
        if transformation is not None:
            self.transformations.append(transformation)


    def apply_transformations(self, data, params=None, updateUoD=False, **kwargs):
        """
        Apply the data transformations for data preprocessing

        :param data: input data
        :param params: transformation parameters
        :param updateUoD:
        :param kwargs:
        :return: preprocessed data
        """

        ndata = data
        if updateUoD:
            if min(data) < 0:
                self.original_min = min(data) * 1.1
            else:
                self.original_min = min(data) * 0.9

            if max(data) > 0:
                self.original_max = max(data) * 1.1
            else:
                self.original_max = max(data) * 0.9

        if len(self.transformations) > 0:
            if params is None:
                params = [ None for k in self.transformations]

            for c, t in enumerate(self.transformations, start=0):
                ndata = t.apply(ndata, params[c], )

        return ndata


    def apply_inverse_transformations(self, data, params=None, **kwargs):
        """
        Apply the data transformations for data postprocessing

        :param data: input data
        :param params: transformation parameters
        :param updateUoD:
        :param kwargs:
        :return: postprocessed data
        """
        if len(self.transformations) > 0:
            if params is None:
                params = [None for k in self.transformations]

            for c, t in enumerate(reversed(self.transformations), start=0):
                ndata = t.inverse(data, params[c], **kwargs)

            return ndata
        else:
            return data


    def get_UoD(self):
        """
        Returns the interval of the known bounds of the universe of discourse (UoD), i. e.,
        the known minimum and maximum values of the time series.

        :return: A set with the lower and the upper bounds of the UoD
        """
        if self.partitioner is not None:
            return (self.partitioner.min, self.partitioner.max)
        else:
            return (self.original_min, self.original_max)


    def offset(self):
        """
        Returns the number of lags to skip in the input test data in order to synchronize it with
        the forecasted values given by the predict function. This is necessary due to the order of the
        model, among other parameters.

        :return: An integer with the number of lags to skip
        """
        if self.is_high_order:
            return self.max_lag
        else:
            return 1


    def __str__(self):
        """
        String representation of the model

        :return: a string containing the name of the model and the learned rules
        (if the model was already trained)
        """

        tmp = self.name + ":\n"
        if self.partitioner.type == 'common':
            for r in sorted(self.flrgs, key=lambda key: self.flrgs[key].get_midpoint(self.partitioner.sets)):
                tmp = "{0}{1}\n".format(tmp, str(self.flrgs[r]))
        else:
            for r in self.flrgs:
                tmp = "{0}{1}\n".format(tmp, str(self.flrgs[r]))
        return tmp

    def __len__(self):
        """
        The length (number of rules) of the model

        :return: number of rules
        """
        return len(self.flrgs)

    def len_total(self):
        """
        Total length of the model, adding the number of terms in all rules

        :return:
        """
        return sum([len(k) for k in self.flrgs])


    def reset_calculated_values(self):
        """
        Reset all pre-calculated values on the FLRG's

        :return:
        """

        for flrg in self.flrgs.keys():
            self.flrgs[flrg].reset_calculated_values()


    def append_log(self,operation, value):
        pass

In [ ]:
import numpy as np
from pyFTS.common import FuzzySet


class FLR(object):
    """
    Fuzzy Logical Relationship

    Represents a temporal transition of the fuzzy set LHS on time t for the fuzzy set RHS on time t+1.
    """
    def __init__(self, LHS, RHS):
        """
        Creates a Fuzzy Logical Relationship
        """
        self.LHS = LHS
        """Left Hand Side fuzzy set"""
        self.RHS = RHS
        """Right Hand Side fuzzy set"""

    def __str__(self):
        return str(self.LHS) + " -> " + str(self.RHS)



class IndexedFLR(FLR):
    """Season Indexed Fuzzy Logical Relationship"""
    def __init__(self, index, LHS, RHS):
        """
        Create a Season Indexed Fuzzy Logical Relationship
        """
        super(IndexedFLR, self).__init__(LHS, RHS)
        self.index = index
        """seasonal index"""

    def __str__(self):
        return str(self.index) + ": "+ str(self.LHS) + " -> " + str(self.RHS)



def generate_high_order_recurrent_flr(fuzzyData):
    """
    Create a ordered FLR set from a list of fuzzy sets with recurrence

    :param fuzzyData: ordered list of fuzzy sets
    :return: ordered list of FLR
    """
    flrs = []
    for i in np.arange(1,len(fuzzyData)):
        lhs = fuzzyData[i - 1]
        rhs = fuzzyData[i]
        if isinstance(lhs, list) and isinstance(rhs, list):
            for l in lhs:
                for r in rhs:
                    tmp = FLR(l, r)
                    flrs.append(tmp)
        else:
            tmp = FLR(lhs,rhs)
            flrs.append(tmp)
    return flrs



def generate_recurrent_flrs(fuzzyData, steps = 1):
    """
    Create a ordered FLR set from a list of fuzzy sets with recurrence

    :param fuzzyData: ordered list of fuzzy sets
    :param steps: the number of steps ahead on the right side of FLR
    :return: ordered list of FLR
    """
    _tmp_steps = steps - 1
    flrs = []
    for i in np.arange(1,len(fuzzyData) - _tmp_steps):
        lhs = [fuzzyData[i - 1]]
        rhs = [fuzzyData[i+_tmp_steps]]
        for l in np.array(lhs).flatten():
            for r in np.array(rhs).flatten():
                tmp = FLR(l, r)
                flrs.append(tmp)
    return flrs



def generate_non_recurrent_flrs(fuzzyData, steps = 1):
    """
    Create a ordered FLR set from a list of fuzzy sets without recurrence

    :param fuzzyData: ordered list of fuzzy sets
    :return: ordered list of FLR
    """
    flrs = generate_recurrent_flrs(fuzzyData, steps=steps)
    tmp = {}
    for flr in flrs: tmp[str(flr)] = flr
    ret = [value for key, value in tmp.items()]
    return ret



def generate_indexed_flrs(sets, indexer, data, transformation=None, alpha_cut=0.0):
    """
    Create a season-indexed ordered FLR set from a list of fuzzy sets with recurrence

    :param sets: fuzzy sets
    :param indexer: seasonality indexer 
    :param data: original data
    :return: ordered list of FLR 
    """
    flrs = []
    index = indexer.get_season_of_data(data)
    ndata = indexer.get_data(data)
    if transformation is not None:
        ndata = transformation.apply(ndata)
    for k in np.arange(1,len(ndata)):
        lhs = FuzzySet.fuzzyfy_series([ndata[k - 1]], sets, method='fuzzy',alpha_cut=alpha_cut)
        rhs = FuzzySet.fuzzyfy_series([ndata[k]], sets, method='fuzzy',alpha_cut=alpha_cut)
        season = index[k]
        for _l in np.array(lhs).flatten():
            for _r in np.array(rhs).flatten():
                flr = IndexedFLR(season,_l,_r)
                flrs.append(flr)
    return flrs

In [ ]:
import numpy as np


class FLRG(object):
    """
    Fuzzy Logical Relationship Group

    Group a set of FLR's with the same LHS. Represents the temporal patterns for time t+1 (the RHS fuzzy sets)
    when the LHS pattern is identified on time t.
    """

    def __init__(self, order, **kwargs):
        self.LHS = None
        """Left Hand Side of the rule"""
        self.RHS = None
        """Right Hand Side of the rule"""
        self.order = order
        """Number of lags on LHS"""
        self.midpoint = None
        self.lower = None
        self.upper = None
        self.key = None

    def append_rhs(self, set, **kwargs):
        pass


    def get_key(self):
        """Returns a unique identifier for this FLRG"""
        if self.key is None:
            if isinstance(self.LHS, (list, set)):
                names = [c for c in self.LHS]
            elif isinstance(self.LHS, dict):
                names = [self.LHS[k] for k in self.LHS.keys()]
            else:
                names = [self.LHS]

            self.key = ""

            for n in names:
                if len(self.key) > 0:
                    self.key += ","
                self.key += n
        return self.key


    def get_membership(self, data, sets):
        """
        Returns the membership value of the FLRG for the input data

        :param data: input data
        :param sets: fuzzy sets
        :return: the membership value
        """
        ret = 0.0
        if isinstance(self.LHS, (list, set)):
            if len(self.LHS) == len(data):
                ret = np.nanmin([sets[self.LHS[ct]].membership(dat) for ct, dat in enumerate(data)])
        else:
            ret = sets[self.LHS].membership(data)
        return ret


    def get_midpoint(self, sets):
        """
        Returns the midpoint value for the RHS fuzzy sets

        :param sets: fuzzy sets
        :return: the midpoint value
        """
        if self.midpoint is None:
            self.midpoint = np.nanmean(self.get_midpoints(sets))
        return self.midpoint


    def get_midpoints(self, sets):
        if isinstance(self.RHS, (list, set)):
            return np.array([sets[s].centroid for s in self.RHS])
        elif isinstance(self.RHS, dict):
            return np.array([sets[s].centroid for s in self.RHS.keys()])


    def get_lower(self, sets):
        """
        Returns the lower bound value for the RHS fuzzy sets

        :param sets: fuzzy sets
        :return: lower bound value
        """
        if self.lower is None:
            if isinstance(self.RHS, list):
                self.lower = min([sets[rhs].lower for rhs in self.RHS])
            elif isinstance(self.RHS, dict):
                self.lower = min([sets[self.RHS[s]].lower for s in self.RHS.keys()])
        return self.lower


    def get_upper(self, sets):
        """
        Returns the upper bound value for the RHS fuzzy sets

        :param sets: fuzzy sets
        :return: upper bound value
        """
        if self.upper is None:
            if isinstance(self.RHS, list):
                self.upper = max([sets[rhs].upper for rhs in self.RHS])
            elif isinstance(self.RHS, dict):
                self.upper = max([sets[self.RHS[s]].upper for s in self.RHS.keys()])
        return self.upper


    def __len__(self):
        return len(self.RHS)

    def reset_calculated_values(self):
        self.midpoint = None
        self.upper = None
        self.lower = None

In [ ]:
from pyFTS.common import fts, FuzzySet, FLR, Membership
from pyFTS.partitioners import Grid
from pyFTS.models.multivariate import FLR as MVFLR, common, flrg as mvflrg
from itertools import product
from types import LambdaType
from copy import deepcopy

import numpy as np
import pandas as pd


def product_dict(**kwargs):
    """
    Code by Seth Johnson
    :param kwargs:
    :return:
    """
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in product(*vals):
        yield dict(zip(keys, instance))



class MVFTS(fts.FTS):
    """
    Multivariate extension of Chen's ConventionalFTS method
    """
    def __init__(self, **kwargs):
        super(MVFTS, self).__init__(**kwargs)
        self.explanatory_variables = kwargs.get('explanatory_variables',[])
        self.target_variable = kwargs.get('target_variable',None)
        self.flrgs = {}
        self.is_multivariate = True
        self.shortname = "MVFTS"
        self.name = "Multivariate FTS"
        self.uod_clip = False

    def append_transformation(self, transformation, **kwargs):
        if not transformation.is_multivariate:
            raise Exception('The transformation is not multivariate')
        self.transformations.append(transformation)
        self.transformations_param.append(kwargs)


    def append_variable(self, var):
        """
        Append a new endogenous variable to the model

        :param var: variable object
        :return:
        """
        self.explanatory_variables.append(var)


    def format_data(self, data):
        ndata = {}
        for var in self.explanatory_variables:
            ndata[var.name] = var.partitioner.extractor(data[var.data_label])

        return ndata


    def apply_transformations(self, data, params=None, updateUoD=False, **kwargs):
        ndata = data.copy(deep=True)
        for ct, transformation in enumerate(self.transformations):
            ndata = transformation.apply(ndata, **self.transformations_param[ct])

        for var in self.explanatory_variables:
            try:
                values = ndata[var.data_label].values #if isinstance(ndata, pd.DataFrame) else ndata[var.data_label]
                if self.uod_clip and var.partitioner.type == 'common':
                    ndata[var.data_label] = np.clip(values,
                                                    var.partitioner.min, var.partitioner.max)

                ndata[var.data_label] = var.apply_transformations(values)
            except:
                pass

        return ndata


    def generate_lhs_flrs(self, data):
        flrs = []
        lags = {}
        for vc, var in enumerate(self.explanatory_variables):
            data_point = data[var.name]
            lags[var.name] = common.fuzzyfy_instance(data_point, var, tuples=False)

        for path in product_dict(**lags):
            flr = MVFLR.FLR()

            flr.LHS = path

            #for var, fset in path.items():
            #    flr.set_lhs(var, fset)

            if len(flr.LHS.keys()) == len(self.explanatory_variables):
                flrs.append(flr)

        return flrs


    def generate_flrs(self, data):
        flrs = []
        for ct in np.arange(1, len(data.index)):
            ix = data.index[ct-1]
            data_point = self.format_data( data.loc[ix] )

            tmp_flrs = self.generate_lhs_flrs(data_point)

            target_ix = data.index[ct]
            target_point = data[self.target_variable.data_label][target_ix]
            target = common.fuzzyfy_instance(target_point, self.target_variable)

            for flr in tmp_flrs:
                for v, s in target:
                    new_flr = deepcopy(flr)
                    new_flr.set_rhs(s)
                    flrs.append(new_flr)

        return flrs


    def generate_flrg(self, flrs):
        for flr in flrs:
            flrg = mvflrg.FLRG(lhs=flr.LHS)

            if flrg.get_key() not in self.flrgs:
                self.flrgs[flrg.get_key()] = flrg

            self.flrgs[flrg.get_key()].append_rhs(flr.RHS)


    def train(self, data, **kwargs):

        ndata = self.apply_transformations(data)

        flrs = self.generate_flrs(ndata)
        self.generate_flrg(flrs)


    def forecast(self, data, **kwargs):
        ret = []
        ndata = self.apply_transformations(data)
        c = 0
        for index, row in ndata.iterrows() if isinstance(ndata, pd.DataFrame) else enumerate(ndata):
            data_point = self.format_data(row)
            flrs = self.generate_lhs_flrs(data_point)
            mvs = []
            mps = []
            for flr in flrs:
                flrg = mvflrg.FLRG(lhs=flr.LHS)
                if flrg.get_key() not in self.flrgs:
                    #Naïve approach is applied when no rules were found
                    if self.target_variable.name in flrg.LHS:
                        fs = flrg.LHS[self.target_variable.name]
                        fset = self.target_variable.partitioner.sets[fs]
                        mp = fset.centroid
                        mv = fset.membership(data_point[self.target_variable.name])
                        mvs.append(mv)
                        mps.append(mp)
                    else:
                        mvs.append(0.)
                        mps.append(0.)
                else:
                    _flrg = self.flrgs[flrg.get_key()]
                    mvs.append(_flrg.get_membership(data_point, self.explanatory_variables))
                    mps.append(_flrg.get_midpoint(self.target_variable.partitioner.sets))

            mv = np.array(mvs)
            mp = np.array(mps)

            ret.append(np.dot(mv,mp.T)/np.nansum(mv))

        ret = self.target_variable.apply_inverse_transformations(ret,
                                                           params=data[self.target_variable.data_label].values)
        return ret


    def forecast_ahead(self, data, steps, **kwargs):
        generators = kwargs.get('generators',None)

        if generators is None:
            raise Exception('You must provide parameter \'generators\'! generators is a dict where the keys' +
                            ' are the dataframe column names (except the target_variable) and the values are ' +
                            'lambda functions that accept one value (the actual value of the variable) '
                            ' and return the next value or trained FTS models that accept the actual values and '
                            'forecast new ones.')

        ndata = self.apply_transformations(data)

        start = kwargs.get('start_at', 0)

        ndata = ndata.iloc[start: start + self.max_lag]
        ret = []
        for k in np.arange(0, steps):
            sample = ndata.iloc[-self.max_lag:]
            tmp = self.forecast(sample, **kwargs)

            if isinstance(tmp, (list, np.ndarray)):
                tmp = tmp[-1]

            ret.append(tmp)

            new_data_point = {}

            for data_label in generators.keys():
                if data_label != self.target_variable.data_label:
                    if isinstance(generators[data_label], LambdaType):
                        last_data_point = ndata.loc[ndata.index[-1]]
                        new_data_point[data_label] = generators[data_label](last_data_point[data_label])
                    elif isinstance(generators[data_label], fts.FTS):
                        gen_model = generators[data_label]
                        last_data_point = sample.iloc[-gen_model.order:]

                        if not gen_model.is_multivariate:
                            last_data_point = last_data_point[data_label].values

                        new_data_point[data_label] = gen_model.forecast(last_data_point)[0]

            new_data_point[self.target_variable.data_label] = tmp

            ndata = ndata.append(new_data_point, ignore_index=True)

        return ret[-steps:]


    def forecast_interval(self, data, **kwargs):
        ret = []
        ndata = self.apply_transformations(data)
        c = 0
        for index, row in ndata.iterrows() if isinstance(ndata, pd.DataFrame) else enumerate(ndata):
            data_point = self.format_data(row)
            flrs = self.generate_lhs_flrs(data_point)
            mvs = []
            ups = []
            los = []
            for flr in flrs:
                flrg = mvflrg.FLRG(lhs=flr.LHS)
                if flrg.get_key() not in self.flrgs:
                    #Naïve approach is applied when no rules were found
                    if self.target_variable.name in flrg.LHS:
                        fs = flrg.LHS[self.target_variable.name]
                        fset = self.target_variable.partitioner.sets[fs]
                        up = fset.upper
                        lo = fset.lower
                        mv = fset.membership(data_point[self.target_variable.name])
                        mvs.append(mv)
                        ups.append(up)
                        los.append(lo)
                    else:
                        mvs.append(0.)
                        ups.append(0.)
                        los.append(0.)
                else:
                    _flrg = self.flrgs[flrg.get_key()]
                    mvs.append(_flrg.get_membership(data_point, self.explanatory_variables))
                    ups.append(_flrg.get_upper(self.target_variable.partitioner.sets))
                    los.append(_flrg.get_lower(self.target_variable.partitioner.sets))

            mv = np.array(mvs)
            up = np.dot(mv, np.array(ups).T) / np.nansum(mv)
            lo = np.dot(mv, np.array(los).T) / np.nansum(mv)

            ret.append([lo, up])

        ret = self.target_variable.apply_inverse_transformations(ret,
                                                           params=data[self.target_variable.data_label].values)
        return ret

    def forecast_ahead_interval(self, data, steps, **kwargs):
        generators = kwargs.get('generators', None)

        if generators is None:
            raise Exception('You must provide parameter \'generators\'! generators is a dict where the keys' +
                            ' are the dataframe column names (except the target_variable) and the values are ' +
                            'lambda functions that accept one value (the actual value of the variable) '
                            ' and return the next value or trained FTS models that accept the actual values and '
                            'forecast new ones.')

        ndata = self.apply_transformations(data)

        start = kwargs.get('start_at', 0)

        ret = []
        ix = ndata.index[start: start + self.max_lag]
        lo = ndata.loc[ix] #[ndata.loc[k] for k in ix]
        up = ndata.loc[ix] #[ndata.loc[k] for k in ix]
        for k in np.arange(0, steps):
            tmp_lo = self.forecast_interval(lo[-self.max_lag:], **kwargs)[0]
            tmp_up = self.forecast_interval(up[-self.max_lag:], **kwargs)[0]

            ret.append([min(tmp_lo), max(tmp_up)])

            new_data_point_lo = {}
            new_data_point_up = {}

            for data_label in generators.keys():
                if data_label != self.target_variable.data_label:
                    if isinstance(generators[data_label], LambdaType):
                        last_data_point_lo = lo.loc[lo.index[-1]]
                        new_data_point_lo[data_label] = generators[data_label](last_data_point_lo[data_label])
                        last_data_point_up = up.loc[up.index[-1]]
                        new_data_point_up[data_label] = generators[data_label](last_data_point_up[data_label])
                    elif isinstance(generators[data_label], fts.FTS):
                        model = generators[data_label]
                        last_data_point_lo = lo.loc[lo.index[-model.order:]]
                        last_data_point_up = up.loc[up.index[-model.order:]]

                        if not model.is_multivariate:
                            last_data_point_lo = last_data_point_lo[data_label].values
                            last_data_point_up = last_data_point_up[data_label].values

                        new_data_point_lo[data_label] = model.forecast(last_data_point_lo)[0]
                        new_data_point_up[data_label] = model.forecast(last_data_point_up)[0]

            new_data_point_lo[self.target_variable.data_label] = min(tmp_lo)
            new_data_point_up[self.target_variable.data_label] = max(tmp_up)

            lo = lo.append(new_data_point_lo, ignore_index=True)
            up = up.append(new_data_point_up, ignore_index=True)

        return ret[-steps:]


    def clone_parameters(self, model):
        super(MVFTS, self).clone_parameters(model)

        self.explanatory_variables = model.explanatory_variables
        self.target_variable = model.target_variable


    def __str__(self):
        _str = self.name + ":\n"
        for k in self.flrgs.keys():
            _str += str(self.flrgs[k]) + "\n"

        return _str

In [ ]:
# Source code for pyFTS.models.multivariate.FLR

class FLR(object):
    """Multivariate Fuzzy Logical Relationship"""

    def __init__(self):
        """
        Creates a Fuzzy Logical Relationship
        :param LHS: Left Hand Side fuzzy set
        :param RHS: Right Hand Side fuzzy set
        """
        self.LHS = {}
        self.RHS = None

    def set_lhs(self, var, set):
        self.LHS[var] = set


    def set_rhs(self, set):
        self.RHS = set


    def __str__(self):
        return "{} -> {}".format([k for k in self.LHS.values()], self.RHS)

In [ ]:
# Source code for pyFTS.models.multivariate.flrg


import numpy as np
from pyFTS.common import flrg as flg


class FLRG(flg.FLRG):
    """
    Multivariate Fuzzy Logical Rule Group
    """

    def __init__(self, **kwargs):
        super(FLRG,self).__init__(0,**kwargs)
        self.order = kwargs.get('order', 1)
        self.LHS = kwargs.get('lhs', {})
        self.RHS = set()

    def set_lhs(self, var, fset):
        if self.order == 1:
            self.LHS[var] = fset
        else:
            if var not in self.LHS:
                self.LHS[var] = []
            self.LHS[var].append(fset)


    def append_rhs(self, fset, **kwargs):
        self.RHS.add(fset)


    def get_membership(self, data, variables):
        mvs = []
        for var in variables:
            s = self.LHS[var.name]
            mvs.append(var.partitioner.sets[s].membership(data[var.name]))

        return np.nanmin(mvs)


    def get_lower(self, sets):
        if self.lower is None:
            self.lower = min([sets[rhs].lower for rhs in self.RHS])

        return self.lower


    def get_upper(self, sets):
        if self.upper is None:
            self.upper = max([sets[rhs].upper for rhs in self.RHS])

        return self.upper


    def __str__(self):
        _str = ""
        for k in self.RHS:
            _str += "," if len(_str) > 0 else ""
            _str += k

        return self.get_key() + " -> " + _str


In [ ]:
# Source code for pyFTS.models.multivariate.partitioner

from pyFTS.partitioners import partitioner
from pyFTS.models.multivariate.common import MultivariateFuzzySet, fuzzyfy_instance_clustered
from itertools import product
from scipy.spatial import KDTree
import numpy as np
import pandas as pd
import sys


class MultivariatePartitioner(partitioner.Partitioner):
    """
    Base class for partitioners which use the MultivariateFuzzySet
    """

    def __init__(self, **kwargs):
        super(MultivariatePartitioner, self).__init__(name="MultivariatePartitioner", preprocess=False, **kwargs)

        self.type = 'multivariate'
        self.sets = {}
        self.kdtree = None
        self.index = {}
        self.explanatory_variables = kwargs.get('explanatory_variables', [])
        self.target_variable = kwargs.get('target_variable', None)
        self.neighbors = kwargs.get('neighbors', 2)
        self.optimize = kwargs.get('optimize', True)
        if self.optimize:
            self.count = {}
        data = kwargs.get('data', None)
        self.build(data)
        self.uod = {}

        self.min = self.target_variable.partitioner.min
        self.max = self.target_variable.partitioner.max


    def format_data(self, data):
        ndata = {}
        for var in self.explanatory_variables:
            ndata[var.name] = var.partitioner.extractor(data[var.data_label])

        return ndata


    def build(self, data):
        pass


    def append(self, fset):
        self.sets[fset.name] = fset


    def prune(self):

        if not self.optimize:
            return

        for fset in [fs for fs in self.sets.keys()]:
            if fset not in self.count:
                fs = self.sets.pop(fset)
                del (fs)

        self.build_index()


    def search(self, data, **kwargs):
        """
        Perform a search for the nearest fuzzy sets of the point 'data'. This function were designed to work with several
        overlapped fuzzy sets.

        :param data: the value to search for the nearest fuzzy sets
        :param type: the return type: 'index' for the fuzzy set indexes or 'name' for fuzzy set names.
        :return: a list with the nearest fuzzy sets
        """

        if self.kdtree is None:
            self.build_index()

        type = kwargs.get('type', 'index')

        ndata = [data[k.name] for k in self.explanatory_variables]
        _, ix = self.kdtree.query(ndata, self.neighbors)

        if not isinstance(ix, (list, np.ndarray)):
            ix = [ix]

        if self.optimize:
            tmp = []
            for k in ix:
                tmp.append(self.index[k])
                self.count[self.index[k]] = 1

        if type == 'name':
            return [self.index[k] for k in ix]
        elif type == 'index':
            return sorted(ix)


    def fuzzyfy(self, data, **kwargs):
        return fuzzyfy_instance_clustered(data, self, **kwargs)


    def change_target_variable(self, variable):
        self.target_variable = variable
        for fset in self.sets.values():
            fset.set_target_variable(variable)
        self.min = variable.partitioner.min
        self.max = variable.partitioner.max


    def build_index(self):

        midpoints = []

        self.index = {}

        for ct, fset in enumerate(self.sets.values()):
            mp = []
            for vr in self.explanatory_variables:
                mp.append(fset.sets[vr.name].centroid)
            midpoints.append(mp)
            self.index[ct] = fset.name


        sys.setrecursionlimit(100000)

        self.kdtree = KDTree(midpoints)

        sys.setrecursionlimit(1000)


In [ ]:
# Source code for pyFTS.models.multivariate.variable

import pandas as pd
from pyFTS.common import fts, FuzzySet, FLR, Membership, tree
from pyFTS.partitioners import Grid
from pyFTS.models.multivariate import FLR as MVFLR


class Variable:
    """
    A variable of a fuzzy time series multivariate model. Each variable contains its own
    transformations and partitioners.
    """
    def __init__(self, name, **kwargs):
        """

        :param name:
        :param \**kwargs: See below

        :Keyword Arguments:
            * *alias* -- Alternative name for the variable
        """
        self.name = name
        """A string with the name of the variable"""
        self.alias = kwargs.get('alias', self.name)
        """A string with the alias of the variable"""
        self.data_label = kwargs.get('data_label', self.name)
        """A string with the column name on DataFrame"""
        self.type = kwargs.get('type', 'common')
        self.data_type = kwargs.get('data_type', None)
        """The type of the data column on Pandas Dataframe"""
        self.mask = kwargs.get('mask', None)
        """The mask for format the data column on Pandas Dataframe"""
        self.transformation = kwargs.get('transformation', None)
        """Pre processing transformation for the variable"""
        self.transformation_params = kwargs.get('transformation_params', None)
        self.partitioner = None
        """UoD partitioner for the variable data"""
        self.alpha_cut = kwargs.get('alpha_cut', 0.0)
        """Minimal membership value to be considered on fuzzyfication process"""


        if kwargs.get('data', None) is not None:
            self.build(**kwargs)

    def build(self, **kwargs):
        """

        :param kwargs:
        :return:
        """
        fs = kwargs.get('partitioner', Grid.GridPartitioner)
        mf = kwargs.get('func', Membership.trimf)
        np = kwargs.get('npart', 10)
        data = kwargs.get('data', None)
        kw = kwargs.get('partitioner_specific', {})
        self.partitioner = fs(data=data[self.data_label].values, npart=np, func=mf,
                              transformation=self.transformation, prefix=self.alias,
                              variable=self.name, **kw)

        self.partitioner.name = self.name + " " + self.partitioner.name


    def apply_transformations(self, data, **kwargs):

        if kwargs.get('params', None) is not None:
            self.transformation_params = kwargs.get('params', None)

        if self.transformation is not None:
            return self.transformation.apply(data, self.transformation_params)

        return data


    def apply_inverse_transformations(self, data, **kwargs):

        if kwargs.get('params', None) is not None:
            self.transformation_params = kwargs.get('params', None)

        if self.transformation is not None:
            return self.transformation.inverse(data, self.transformation_params)

        return data


    def __str__(self):
        return self.name
